In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('best_m.pt')

# Open the video file
video_path = "sample.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps*4, (frame_width // 2, frame_height // 2))  # fps*4 pour accélérer la vidéo

# Set the desired frame rate (increase to speed up)
frame_rate = 12  # Augmentez cette valeur pour accélérer davantage

i = 0
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        if i % frame_rate == 0:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Resize the frame (example: half size)
            annotated_frame = cv2.resize(annotated_frame, (frame_width // 2, frame_height // 2))

            # Write the annotated frame to the output video
            out.write(annotated_frame)

            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1000 // frame_rate) & 0xFF == ord("q"):
                break
    else:
        # Break the loop if the end of the video is reached
        break
    i += 1

# Release the video capture and writer objects, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()
